Walrasian_auctioneer_or_criers
Not an auction market, simply a market where someone–-the crier–-proposes prices, following a trial and error process, to search for an equlibrium. See https://en.wikipedia.org/wiki/Walrasian_auction

paper utile https://air.unimi.it/bitstream/2434/43747/1/walras_lombardo_07fin.pdf

localmente file walras_lombardo_07fin.pdf
Equilibrio, disequilibrio e tempo in Walras 

Errore traduzione, vedere **Walrasian auction** https://en.wikipedia.org/wiki/Walrasian_auction

Using **[Swarm (experimental, educational)](https://github.com/openai/swarm/blob/main/README.md)** con **[Orchestrating Agents: Routines and Handoffs](https://cookbook.openai.com/examples/orchestrating_agents)**

altri: [LangGraph vs CrewAI vs OpenAI Swarm: Which AI Agent Framework to Choose?](https://oyelabs.com/langgraph-vs-crewai-vs-openai-swarm-ai-agent-framework/)

If missing  
! pip install git+https://github.com/openai/swarm.git  

In [1]:
from swarm import Swarm, Agent
from dotenv import load_dotenv
import tool
import os

In [2]:
load_dotenv()
os.getenv('OPENAI_API_KEY');

In [3]:
client = Swarm()

In [4]:
#client.client.api_key

In [5]:
client.client.api_key = tool.tool()

Enter code  ········


In [6]:
import random as r

def setRandomPrice() -> float:
    """generate a random price"""
    return 100 * r.random()

## Introducing the crier, suggesting how to operate, asking very concise replies, price level set randomly

In [7]:
def instructions(context_variables):
   price_level = context_variables["price_level"]
   rules = "1. Use the function setRandomPrice " +\
           "to set your reservation price as a random price and report "+\
           "it as a pure number without money specification. "+\
           "2. Report your id using the sequence buyerSeller"+str(i)+". "+\
           "3. Report if you buy or sell comparing you reservation price and the price level, "+\
           "considering that if your reservation price is greater that the price level is convenient to buy "+\
           "and that in the opposite case is convenient to sell. "+\
           "If your reservation price is very close to the price level, do nothing "+\
           f"The price level is {price_level}"
 
   return rules

In [8]:
lA=[]

for i in range(10):
 lA.append(
    Agent(
    name="Agent buying-selling",
    instructions = instructions,
    functions= [setRandomPrice]
    )
 )

In [9]:
def instructions2(context_variables):
    global pl
    pl=100 * r.random()
    print("price level ",pl)
    rules = "Report your id as 'crier'"
    return rules  

In [10]:
crier = Agent(
    name="Agent crier",
    instructions = instructions2
    )

In [11]:
messages = [{"role": "user", "content": "Report your id"}]
response = client.run(
                       agent=crier, 
                       messages=messages)
print(response.messages[-1]["content"])

price level  48.58351481483759
My id is 'crier'.


In [ ]:
for i in range(10):
 messages = [{"role": "user", "content": "Report your id, what is your reservation price and that do you do;"+
                                         "report in a very concise way"}]
 response = client.run(
                       agent=lA[i], 
                       messages=messages,
                       context_variables={"price_level": pl})
 print(response.messages[-1]["content"])

ID: buyerSeller0, Reservation Price: 1.34, Action: Sell
Id: buyerSeller1, Reservation Price: 45.36360302649334, Action: Sell
ID: buyerSeller2, Reservation Price: 16.29574918821932, Action: Sell
- ID: buyerSeller3
- Reservation Price: 98.34
- Action: Buy (Reservation price is greater than the price level 48.58)
ID: buyerSeller4, Reservation Price: 78.00820529811494, Action: Buy
ID: buyerSeller5, Reservation Price: 35.24680514165578, Action: Sell
Id: buyerSeller6, Reservation Price: 95.73873912652347, Action: Buy
